<a href="https://colab.research.google.com/github/MarioCastilloM/Activos_Derivados/blob/main/Options_Valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img style="float: right;" align="center" src="https://www.udp.cl/cms/wp-content/uploads/2021/06/UDP_LogoRGB_2lineas_Color_SinFondo.png">

# Trabajo Activos Derivados

<!-- ![picture](https://www.udp.cl/cms/wp-content/uploads/2021/06/UDP_LogoRGB_2lineas_Color_SinFondo.png) -->

## Integrantes

-  Mario Castillo
-  Felipe Gálvez

## Introducción

A continuación se presenta la valoración de opciones con parámetros similares por distintos métodos, con la finalidad de comparar los resultados y evaluar la consistencia de estos. Adicionalmente, se presentan valoraciones de opciones americanas y asiáticas.

Para realizar este proyecto, se decidió crear la clase ***Options*** en Python, de modo que la programación, en términos de modificar fórmulas y agregar procedimientos o funciones no implicara una reconstrucción del código. La clase ***Options*** considera los métodos requeridos para este trabajo, a saber:

- Método Black-Scholes-Merton con y sin dividendos
- Valoración por Árbol Binomial para opciones Europeas
- Valoracion por Árbol Binomial para opciones Americanas
- Simulaciones de Montecarlo log-normal, árbol binomial y para opciones asiáticas con strike-promedio

Junto con el cálculo del precio en cada uno de los casos, se presenta una breve descripción del método utilizado.

In [16]:
class Options:
  def __init__(self, s0, X, rf, sigma, div, T, n, option):
    self.s0 = s0
    self.X = X
    self.rf = rf
    self.sigma = sigma
    self.div = div
    self.T = T
    self.option = option
    self.n = n

  def priceBSM(self):
    import numpy as np
    from scipy.stats import norm
    if self.div == 0:
      d1 = (np.log((self.s0)/self.X) + (self.rf + (self.sigma**2)/2) * self.T)/(self.sigma*np.sqrt(self.T))
      d2 = d1 - self.sigma * np.sqrt(self.T)

      if self.option == "call":
        price = (self.s0)*norm.cdf(d1) - self.X*np.exp(-self.rf*self.T)*norm.cdf(d2)
      elif self.option == "put":
        price = self.X*np.exp(-self.rf*self.T)*norm.cdf(-d2) - (self.s0)*norm.cdf(-d1)
      else:
        print("ingrese opción válida")
        price = 0
    else:
      vpd = self.div
      d1 = (np.log((self.s0 - vpd)/self.X) + (self.rf + self.sigma**2/2)*self.T)/(self.sigma*np.sqrt(self.T))
      d2 = d1 - self.sigma * np.sqrt(self.T)

      if self.option == "call":
        price = (self.s0 - vpd)*norm.cdf(d1) - self.X*np.exp(-self.rf*self.T)*norm.cdf(d2)
      elif self.option == "put":
        price = self.X*np.exp(-self.rf*self.T)*norm.cdf(-d2) - (self.s0 - vpd)*norm.cdf(-d1)
      else:
        print("ingrese opción válida")
        price = 0
    return price

  def parity(self, price):
    import numpy as np
    if self.div == 0:
      if self.option == "call":
        par = price + self.X*(1+self.rf)**-self.T - self.s0
      elif self.option == "put":
        par = price - self.X*(1+self.rf)**-self.T + self.s0
      else:
        print("ingrese opción válida")
        par = 0
    else:
      vpd = np.exp(-self.div*self.T)
      if self.option == "call":
        vpd = np.exp(-self.div*self.T)
        par = price + self.X*(1+self.rf)**-self.T - self.s0*vpd
      elif self.option == "put":
        par = price - self.X*(1+self.rf)**-self.T + self.s0*vpd
      else:
        print("ingrese opción válida")
        par = 0
    return par
  def binomial_american(self):
    import numpy as np
    u = np.exp(self.sigma * np.sqrt(self.T/self.n))
    d = 1/u
    exp = np.exp(self.rf * self.T/self.n)
    p = (exp - d)/(u - d)
    df = np.zeros((self.n + 1, self.n + 1))
    for i in range(self.n + 1):
      for j in range(1, self.n + 1):
        df[0, 0] = self.s0
        df[i, 0] = 0
        if j >= i:
          df[i, j] = self.s0 * u**(j-i) * d**i
        else:
          df[i, j] = 0
    df1 = np.zeros(df.shape)
    if self.option == "put":
      for j in range(len(df1)-1, -1, -1):
        for i in range(len(df1)):
          if j == len(df1)-1:
            df1[i, j] = max(self.X - df[i, j], 0)
          elif i <= j:
            df1[i, j] = max(((p * df1[i, j+1] + (1-p) * df1[i+1, j+1]))/exp, self.X - df[i, j])
          else:
            df1[i, j] = 0
    elif self.option == "call":
      for j in range(len(df1)-1, -1, -1):
        for i in range(len(df1)):
          if j == len(df1)-1:
            df1[i, j] = max(df[i, j] - self.X, 0)
          elif i <= j:
            df1[i, j] = max(((p * df1[i, j+1] + (1-p) * df1[i+1, j+1]))/exp, df[i, j] - self.X)
          else:
            df1[i, j] = 0
    else:
        print("option type is not valid")
        par = 0
    return df1[0, 0]

  def binomial_european(self):
    import numpy as np
    u = np.exp(self.sigma * np.sqrt(self.T/self.n))
    d = 1/u
    exp = np.exp(self.rf * self.T/self.n)
    p = (exp - d)/(u - d)
    df = np.zeros((self.n + 1, self.n + 1))
    for i in range(self.n + 1):
      for j in range(1, self.n + 1):
        df[0, 0] = self.s0
        df[i, 0] = 0
        if j >= i:
          df[i, j] = self.s0 * u**(j-i) * d**i
        else:
          df[i, j] = 0
    df1 = np.zeros(df.shape)
    if self.option == "put":
      for j in range(len(df1)-1, -1, -1):
        for i in range(len(df1)):
          if j == len(df1)-1:
            df1[i, j] = max(self.X - df[i, j], 0)
          elif i <= j:
            df1[i, j] = ((p * df1[i, j+1] + (1-p) * df1[i+1, j+1]))/exp
          else:
            df1[i, j] = 0
    elif self.option == "call":
      for j in range(len(df1)-1, -1, -1):
        for i in range(len(df1)):
          if j == len(df1)-1:
            df1[i, j] = max(df[i, j] - self.X, 0)
          elif i <= j:
            df1[i, j] = ((p * df1[i, j+1] + (1-p) * df1[i+1, j+1]))/exp
          else:
            df1[i, j] = 0
    else:
        print("option type is not valid")
        par = 0
    return df1[0, 0]

  def monte_carlo_ln(self, J):
    import numpy as np
    e = np.random.normal(0, 1, J)
    op = np.zeros(J)
    for i in range(J):
      st = self.s0 * np.exp((self.rf - ((self.sigma**2) / 2)) * self.T + self.sigma * e[i] * np.sqrt(self.T))
      if self.option == "call":
        op[i] = max(st - self.X, 0) * np.exp(-self.rf * self.T)
      elif self.option == "put":
        op[i] = max(self.X - st, 0) * np.exp(-self.rf * self.T)
      else:
        op[i] = 0
    price = np.mean(op)
    return price, [price - 1.96 * np.std(op) / np.sqrt(J), price + 1.96 * np.std(op) / np.sqrt(J)]
  def monte_carlo_bt(self, J):
    import numpy as np
    delta_t = self.T/self.n
    u = np.exp(self.sigma * np.sqrt(delta_t))
    d = 1/u
    p = (np.exp(self.rf * delta_t) - d)/(u - d)
    op = np.zeros(J)
    for i in range(J):
      probs = np.random.uniform(0, 1, self.n)
      delt = len(probs[probs < (1-p)])
      st = self.s0 * (d**delt) * (u**(self.n - delt))
      if self.option == "call":
        op[i] = max(st - self.X, 0) * np.exp(-self.rf * self.T)
      elif self.option == "put":
        op[i] = max(self.X - st, 0) * np.exp(-self.rf * self.T)
      else:
        op[i] = 0
    price = np.mean(op)
    return price, [price - 1.96 * np.std(op) / np.sqrt(J), price + 1.96 * np.std(op) / np.sqrt(J)]

  def monte_carlo_asian(self, J):
    import numpy as np
    u = np.exp(self.sigma * np.sqrt(self.T/self.n))
    d = 1/u
    exp = np.exp(self.rf * self.T/self.n)
    p = (exp - d)/(u - d)
    op = np.zeros(J)
    s = np.zeros(self.n)
    for j in range(J):
      s[0] = self.s0
      probs = np.random.uniform(0, 1, self.n)
      for i in range(1, self.n):
        if probs[i] >= (1 - p):
          s[i] = s[i-1] * u
        else:
          s[i] = s[i-1] * d
      if self.option == "call":
        op[j] = (max(self.s0 - np.mean(s), 0)) * np.exp(-self.rf * self.T)
      if self.option == "put":
        op[j] = (max(np.mean(s) - self.s0, 0)) * np.exp(-self.rf * self.T)
    price = np.mean(op)
    return price, [price - 1.96 * np.std(op) / np.sqrt(J), price + 1.96 * np.std(op) / np.sqrt(J)]

## Pregunta 1

### Valoración por BSM

El modelo Black, Scholes and Merton es utilizado para la valorización de opciones, a través de la utilización de una ecuación diferencial. Requiere de 5 variables inputs:

- Strike
- Precio actual del subyacente
- Tiempo a la madurez
- Tasa libre de riesgo
- Volatilidad del activo subyacente

Este modelo aplica para opciones europeas, es decir, que no son ejercidas antes de la madurez.

Los supuestos que subyacen al modelo son los siguientes:

- La acción no paga dividendos (*)
- Los mercados se comportan de manera aleatoria
- No hay costos de transacción al comprar la opción
- La tasa libre de riesgo y la volatilidad del activo subyacente son conocidas y constantes.
- La opción valorada es europea y sólo puede ser ejercida a la madurez

(\*) *Es posible valorar opciones sobre acciones que pagan dividendos con una pequeña modificación a las fórmulas del modelo*

In [18]:
p1 = Options(s0 = 50, X = 52, rf = 0.05, sigma=0.3, div=0, T = 2, n = 500, option="put")
pp1 = p1.priceBSM()
print(f"El precio de la opción put europea por el método BSM es {round(pp1, 2)}")

El precio de la opción put europea por el método BSM es 6.76


### Valoración por árbol binomial

Si bien el modelo de Black-Scholes-Merton es uno de los más populares en la valoración de opciones, tiene limitaciones considerables, por ejemplo en la valoración de opciones americanas. Una alternativa a esta metodología es la valoración de la opción a través de un árbol binomial que considera, primero, dos movimientos posibles del precio del activo subyacente, hacia arriba o hacia abajo. Luego de construir el árbol binomial para el precio del activo subyacente, por inducción hacia atrás se construye el árbol del precio de la opción (call), en el período $T$ como $max\{S_T - X, 0\}$, mientras que para el resto de períodos, el precio viene definido por la valoración neutral al riesgo, i.e., $C_t = C_{u,\ t+1}*p^* + C_{d,\ t+1}*(1-p^*)$ en cada nodo, hasta llegar al nodo inicial o $t = 0$. Para realizar la valoración de una opción a través de esta metodología se deben calcular:

- La tasa libre de riesgo
- la volatilidad del activo subyacente
- El strike
- El precio actual del activo subyacente
- Un factor de alza $u = exp(r_f * \sqrt{\frac{T}{n}})$
- un factor de baja $d = 1/u$
- La probabilidad neutral al riesgo $p^*=\frac{exp(rf*\Delta t)-d}{(u-d)}$

In [ ]:
binomial = p1.binomial_european()
print(f"El precio de la opción put europea por el método de árbol binomial es {round(binomial, 2)}")

El precio de la opción put europea por el método de árbol binomial es 6.76


Como se puede observar, los precios obtenidos por ambos métodos, el método de árbol binomial y el método de Black, Scholes and Merton son bastante similares.

## Pregunta 2

### Valoración por simulación de Montecarlo

Hasta aquí hemos presentado dos métodos ampliamente utilizados para la valoración de una opción, sin embargo, con dichos métodos, es difícil proveer soluciones analíticas que incorporen saltos en la volatilidad y el el retorno, de modo que los métodos numéricos, como la simulación de Montecarlo, aparecen como respuesta a estas limitantes.

#### Valoración por simulación de Montecarlo Log-Normal

- Generamos un vector de errores estocásticos $\varepsilon \sim N(0,\ 1)$
- Obtenemos el precio del activo subyacente a la madurez incorporando este error estocástico en el precio a través de la volatilidad
- Calculamos el valor presente del pago de la opción en cada replicación
- Obtenemos el precio promedio de la opción y generamos el intervalo de confianza.

In [ ]:
MC, ICMCln = p1.monte_carlo_ln(J = 30000)
print(f"El precio de la opción put europea por simulación de Montecarlo Log-normal es {round(MC, 2)}\nCon un intervalo de confianza {ICMCln}")

El precio de la opción put europea por simulación de Montecarlo Log-normal es 6.6
Con un intervalo de confianza [6.5033136069392015, 6.69756313641259]


#### Valoración por simulación de árbol binomial

- Calculamos $\Delta t$, $u$, $d$ y $p^*$
- Generamos vector con $n$ probabilidades estocásticas $p^j \sim U(0,\ 1)$ y definimos $\delta$ como el número de probabilidades del vector que son menores a $(1-p^*)$
- Obtenemos el precio del activo a la madurez en cada replicación
- Calculamos el pago de la opción en cada replicación
- Obtenemos el precio de la opción como el promedio del vector de precios de la opción, que contiene el precio de cada replicación y generamos intervalos de confianza

In [ ]:
MCBT, ICMCbt = p1.monte_carlo_bt(J = 30000)
print(f"El precio de la opción put europea por simulación de árbol binomial es {round(MCBT, 2)}\nCon un intervalo de confianza {ICMCbt}")

El precio de la opción put europea por simulación de árbol binomial es 6.7
Con un intervalo de confianza [6.598791853134989, 6.793743591449818]


Ambos métodos nos entregan un resultado similar

## Pregunta 3

### Opciones Exóticas

Este tipo de opciones se diseñan para responder a necesidades específicas de los clientes y se transan en mercados OTC. Una de las más conocidas o utilizadas es la opción asiática.

#### Valoración de Put Asiática por Montecarlo 

- Generamos un vector con n probabilidades tal que $p^j\sim U(0,\ 1)$
- En cada replicación definimos un precio del subyacente imitando la trayectoria que tendría en un arbol binomial, dependiendo de si la probabilidad correspondiente a esa subiteración (sobre el vector de $n$ probabilidades y no de $J$ replicaciones) es mayor o menor que la probabilidad neutral al riesgo
- Calculamos el valor presente de la opción en cada replicación, en este caso, fijando el precio del activo subyacente en $S_0 = 50$ y utilizando como strike el promedio del vector de $n$ precios del activo subyacente para cada replicación
- Una vez que tenemos el vector con $J$ precios de la opción, obtenemos el precio final como el promedio y generamos un intervalo de confianza

In [12]:
asian, ICasian = p1.monte_carlo_asian(J = 30000)
print(f"El precio de la opción put asiática por simulación de Montecarlo es {round(asian, 2)}\nCon un intervalo de confianza {ICasian}")

El precio de la opción put asiática por simulación de Montecarlo es 5.78
Con un intervalo de confianza [5.677030632110896, 5.876654454529757]


## Pregunta 4

### Valoración Put Americana por árbol binomial 

En este caso, se construye igualmente el árbol binomial y el procedimiento de valoración es similar, sin embargo, en cada nodo se debe comparar el precio obtenido a través de la valoración neutral al riesgo, con la diferencia entre el precio del subyacente en ese nodo y el precio strike, con la finalidad de evaluar e cada nodo si es conveniente o no ejercer la opción en ese momento del tiempo

In [20]:
import pandas as pd
binomial_am = p1.binomial_american()
print(f"El precio de la opción put americana por árbol binomial es {round(binomial_am, 2)}")

El precio de la opción put americana por árbol binomial es 7.47


## Pregunta 5

### Valoración de Put Europea con dividendos por árbol binomial y BSM

Al tratarse de una opción europea, el procedimiento del árbol binomial es el mismo descrito en la pregunta 1, sin embargo, al precio $S_0$ le restamos el valor presente de los dividendos. Este, se calcula como el dividendo descontado a la tasa libre de riesgo multiplicada por el plazo a la madurez del dividendo. Asumiendo que la valoración se lleva a cabo en septiembre, el plazo al primer dividendo es 1 año y al segundo dividendo son 2 años, de modo que el valor presente de los dividendos para este caso, vendrán dados por:

$VP_{div} = D_1*exp(-r_f * T_1) + D_2*exp(-r_f*T_2)$

Donde:

- $D_1=1$
- $D_2=1$
- $T_1=1$
- $T_1=2$
- $r_f=0.05$

Por otra parte, el modelo BSM, al incorporar el dividendo en el precio de la opción, considera una ligera modificación en sus 3 fórmulas, sin embargo, consiste en la misma lógica, restarle al precio del subyacente el valor presente de los dividendos:

- $P = X*exp(-r_f*T)*N(-d_2) - [S_0 - D*exp(-r_f*T)]*N(-d_1)$
- $d_1 = \frac{ln((S_0-D*exp(-r_f*T))/X)+(r_f+(\sigma^2)/2)*T}{\sigma\sqrt T}$
- $d_2 = d_1 - \sigma\sqrt T$

In [13]:
import numpy as np
div = (1 * np.exp(-0.05 * 1) + 1 * np.exp(-0.05 * 2))
s0 = 50 - div
p2 = Options(s0 = s0, X = 54, rf = 0.05, sigma=0.3, div=0 , T = 2, n = 500, option="put")
binomial_div = p2.binomial_european()

p3 = Options(s0 = 50, X = 54, rf = 0.05, sigma=0.3, div=div , T = 2, n = 500, option="put")
bsm_div = p3.priceBSM()

print(f"El precio de la opción put europea con pago de dividendos de $1 anual es {round(binomial_div, 2)}\nMientras que por BSM, el precio resultante es de {round(bsm_div, 2)}")

El precio de la opción put europea con pago de dividendos de $1 anual es 8.51
Mientras que por BSM, el precio resultante es de 8.51


Podemos ver que los precios obtenidos por ambos métodos son muy similares

# Conclusión
Finalmente, aplicando distintos métodos de valoración para opciones con iguales parámetros ($S_0$, $X$, $\sigma$, $T$, $n$, $r_f$), podemos ver que los precios obtenidos son sumamente similares, de modo que podemos concluir que con un número de subperíodos y de simulaciones lo suficientemente grandes, el precio obtenido por distintos métodos converge al mismo valor.